In [17]:
import os
import openai
from dotenv import load_dotenv
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.core import load_index_from_storage
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


documents = SimpleDirectoryReader(input_files = ["./testdocument/introduction-to-natural-language-processing.pdf"]).load_data()
document = [Document(text="\n\n".join([doc.text for doc in documents]))]

def get_auto_merging_index(document, index_dir, chunk_sizes):
    
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes = chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(document)
    leaf_nodes = get_leaf_nodes(nodes)

    Settings.llm = OpenAI()
    Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    Settings.node_parser = node_parser
    
    docstore = SimpleDocumentStore()

    # insert nodes into docstore
    docstore.add_documents(nodes)

    # define storage context (will include vector store by default too)
    storage_context = StorageContext.from_defaults(docstore=docstore)
    
    if not os.path.exists(index_dir):
        automerging_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context)
        automerging_index.storage_context.persist(persist_dir=index_dir)
    else:
        automerging_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=index_dir))

    return automerging_index

def get_auto_merging_engine(am_index):
    
    base_retriever = am_index.as_retriever(similarity_top_k=6)
    retriever = AutoMergingRetriever(base_retriever, am_index.storage_context, verbose=True)
    rerank = SentenceTransformerRerank(top_n=2, model="BAAI/bge-reranker-base") 
    auto_merging_engine = RetrieverQueryEngine.from_args(retriever, node_postprocessors=[rerank])
    
    return auto_merging_engine

In [18]:
index_dir = "./automerging_index_3"
am_index_3 = get_auto_merging_index(document, index_dir, chunk_sizes=[2048, 512, 128])
am_engine_3 = get_auto_merging_engine(am_index_3)

In [19]:
window_response_3 = am_engine_3.query(
    "How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?"
)
window_response_3.response

'Corpora are utilized in grammatical research to carry out quantitative studies of grammatical frequency and to test predictions made by formal grammars. Recent advancements in computational power and developments in parsed corpora and analysis tools have enabled researchers to conduct these studies more effectively and efficiently.'

In [20]:
index_dir = "./automerging_index_2"
am_index_2 = get_auto_merging_index(document, index_dir, chunk_sizes=[512, 128])
am_engine_2 = get_auto_merging_engine(am_index_2)

In [21]:
window_response_2 = am_engine_2.query(
    "How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?"
)
window_response_2.response

'Corpora are utilized in grammatical research to conduct quantitative studies on grammatical frequency and to test predictions made by formal grammars. Recent advancements in computational power and developments in parsed corpora and analysis tools have enabled researchers to carry out these studies effectively.'

In [22]:
from trulens_eval import Tru
from trulens_eval import TruLlama
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
from trulens_eval.app import App
import numpy as np

tru = Tru()

# Initialize provider class
provider = OpenAI()

# Select context to be used in feedback. The location of context is app specific.

def get_evaluation_response(rag_engine, app_id, eval_questions):
    
    context = App.select_context(rag_engine)

    # Define a groundedness feedback function
    f_groundedness = (
        Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(context.collect())  # Collect context chunks into a list
        .on_output()
    )

    # Question/answer relevance between overall question and answer.
    f_answer_relevance = (
        Feedback(provider.relevance, name="Answer Relevance")
        .on_input_output()
    )

    # Question/statement relevance between question and each context chunk.
    f_context_relevance = (
        Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(context)
        .aggregate(np.mean)
    )



    # Initialize the recorder
    tru_query_engine_recorder = TruLlama(
        rag_engine,
        app_id= app_id,
        feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])
    
    for question in eval_questions:
        with tru_query_engine_recorder as recording:
            response = rag_engine.query(question)
    records = recording.get()
    
    return records

In [23]:
eval_questions = [
    "Who introduced the notions of finite-state machines and context-free grammar (CFG) to linguistics?",
    "How did linguists test formal rules of grammar according to Chomsky’s approach?",
    "What has contributed to making the vision of computers understanding ordinary language and holding conversations with humans more plausible in the 21st century?",
    "Why is center-embedding significant in grammars, and what does it allow?",
    "What is a corpus in the context of natural language processing, and what are the three broad categories of corpora?",
    "How are corpora used in modern lexicography, and how do they influence dictionary entries?",
    "How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?"
]

records3 = get_evaluation_response(
    am_engine_3,
    app_id='auto merging engine 3 Layer',
    eval_questions = eval_questions
)

records2 = get_evaluation_response(
    am_engine_2,
    app_id='auto merging engine 2 Layer',
    eval_questions = eval_questions
)

display(records3)
display(records2)
tru.run_dashboard()

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings

	instrumenting _retrieve
	instrumenting _aretrieve
instrumenting <class 'llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> for base <class 'llama_index.core.base.base_retriever.BaseRetriever'>
	instrumenting retrieve
	instrumenting _retrieve
	instrumenting _aretrieve
instrumenting <class 'llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> for base <class 'llama_index.core.base.query_pipeline.query.ChainableMixin'>
instrumenting <class 'llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> for base <class 'llama_index.core.prompts.mixin.PromptMixin'>
instrumenting <class 'llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> for base <class 'llama_index.core.instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.core.indices

Record(record_id='record_hash_a90e19d7b1850a609f56a6006cd5506e', app_id='app_hash_b156bcd0de63ea8e01291143ece0db3b', cost=Cost(n_requests=1, n_successful_requests=1, n_completion_requests=1, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=360, n_stream_chunks=0, n_prompt_tokens=309, n_completion_tokens=51, n_cortex_guardrails_tokens=0, cost=0.0005655, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2025, 3, 18, 18, 6, 6, 544243), end_time=datetime.datetime(2025, 3, 18, 18, 6, 36, 551754)), ts=datetime.datetime(2025, 3, 18, 18, 6, 36, 554704), tags='-', meta=None, main_input='How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?', main_output='Corpora are utilized in grammatical research to conduct quantitative studies on grammatical frequency and to test predictions made by formal grammars. Recent advancements in computational power and developments in parsed co

Record(record_id='record_hash_26ebe537f70d7e531842f27d03d11394', app_id='app_hash_6a0ff522e5818a1792721ff43255b6a7', cost=Cost(n_requests=1, n_successful_requests=1, n_completion_requests=1, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=378, n_stream_chunks=0, n_prompt_tokens=309, n_completion_tokens=69, n_cortex_guardrails_tokens=0, cost=0.0006015, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2025, 3, 18, 18, 9, 28, 888406), end_time=datetime.datetime(2025, 3, 18, 18, 9, 54, 867735)), ts=datetime.datetime(2025, 3, 18, 18, 9, 55, 109031), tags='-', meta=None, main_input='How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?', main_output='Corpora are used in grammatical research to conduct quantitative studies on grammatical frequency and to test predictions from formal grammars. The advancements in computational power and tools have enabled researchers to 

Starting dashboard ...
Dashboard already running at path:   Local URL: http://localhost:51178



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [16]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
